In [1]:
%pip install openpyxl



     |████████████████████████████████| 243 kB 1.2 MB/s 
Note: you may need to restart the kernel to use updated packages.


**Data Loading**

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler
from sklearn.decomposition import SparsePCA
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
data = pd.read_excel('../input/upadate2/1-s2.0-S0022231321001368-mmc1.xlsx', index_col=0)

In [3]:
data.head()

,TeO2,ZnO,Na2O,Er2O3,TiO2,BaO,Na2CO3,GeO2,B2O3,MgO,...,PbBr2,PbF2,CaO,P2O5,AL2O3,SIO2,W2,W4,W6,Link
S1,80.721157,11.931813,4.543149,2.803881,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,2.14,2.77,1.42,http://dx.doi.org/10.1016/j.jallcom.2017.07.102
S2,80.651304,11.938790,4.545806,2.805520,0.058579,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,3.08,2.15,1.88,http://dx.doi.org/10.1016/j.jallcom.2017.07.102
S3,80.581370,11.945776,4.548465,2.807162,0.117227,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,3.64,1.27,2.17,http://dx.doi.org/10.1016/j.jallcom.2017.07.102
S4,80.511354,11.952770,4.551128,2.808805,0.175943,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,3.72,1.30,2.22,http://dx.doi.org/10.1016/j.jallcom.2017.07.102
S5,80.441255,11.959771,4.553794,2.810451,0.234728,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,3.49,1.90,2.09,http://dx.doi.org/10.1016/j.jallcom.2017.07.102


In [4]:
df = data.copy()
x = df.iloc[0:,0:27].values.astype(float)
y = df.iloc[0:,27:30].values.astype(float)

use_sc = False

In [5]:
y.shape

(180, 3)

**Sparsity Fixing **

In [6]:
transformer_x = SparsePCA(n_components=25, alpha=1e-03, ridge_alpha=1e-05, max_iter=1000000, tol=1e-05,method='cd')
transformer_x.fit(x)
x= transformer_x.transform(x)


**SVR Model :**
* Scaling with RobustScaler
* 90% training data & 10% testing data, selected Randomly for 1000 iteration 

In [7]:
indexes = np.arange(len(x))

train_percentage=0.9
train_size = int(train_percentage * len(x))
transformerx = RobustScaler().fit(x)
transformery = RobustScaler().fit(y)
#sc_x = MinMaxScaler(feature_range=(-1,1))
sc_y = MinMaxScaler(feature_range=(-1,1))


n = 1000
r= np.empty((3))
RMSE = np.empty((int(n), 3))

for i in range(n):
    indexes = shuffle(indexes, n_samples=len(indexes))
    x = x[indexes]
    y = y[indexes]
    x_train = x[:train_size]
    y_train = y[:train_size]
    x_test = x[train_size:]
    y_test = y[train_size:]

    x_train_raw = x_train
    y_train_raw = y_train
    x_train = transformerx.transform(x_train)
    y_train = transformery.transform(y_train)

    x_test_raw = x_test
    y_test_raw = y_test
    x_test = transformerx.transform(x_test)
    y_test = transformery.transform(y_test)
    
    regressor = MultiOutputRegressor(SVR(kernel='rbf',gamma= 'auto'))
    regressor.fit(x_train, y_train)
    y_pred_train = regressor.predict(x_train).astype(float)
    #y_pred_train = sc_y.inverse_transform(y_pred_train)

    y_pred_test = regressor.predict(x_test).astype(float)
    y_pred_test = transformery.inverse_transform(y_pred_test)

    
    
    
    r=mean_squared_error(y_pred_test, y_test_raw, squared=False,multioutput= 'raw_values')
    print(r)
    RMSE[i] = r
RMSE.mean(axis=0)
    
   







[1.10070496 0.39640357 0.38881489]
[1.45582641 0.62016318 0.48304894]
[1.28947671 0.64875121 0.41389579]
[1.04928121 0.41873363 0.40081522]
[1.43565721 0.52597783 0.26953348]
[1.03301335 0.36937744 0.40079575]
[1.55179349 0.63874926 0.43580593]
[1.07199176 0.41540485 0.36681913]
[1.17723358 0.92506826 0.42036785]
[1.02516197 0.45384522 0.35220907]
[1.3929523  0.48449872 0.40956924]
[1.72940457 0.76408211 0.57843173]
[1.31232329 0.51176455 0.56955493]
[0.90101331 0.32687933 0.31910618]
[0.8369676  0.55269288 0.30496501]
[1.46108212 0.43088069 0.31135402]
[1.45083691 0.77754653 0.5841968 ]
[1.5833335  1.01274717 0.3578929 ]
[1.12007248 0.36879471 0.44033415]
[0.65862159 0.57109598 0.35043648]
[1.20713482 0.49566044 0.48000829]
[0.73788864 0.31144497 0.38448903]
[1.41987731 0.53146091 0.5401847 ]
[1.10555324 0.44333382 0.44502157]
[1.77588095 1.00384171 0.53048357]
[1.05597099 0.4812389  0.35645107]
[1.96847443 0.97751139 0.54154142]
[1.3120687  0.48925464 0.31748112]
[1.10190337 0.382339

array([1.3131393 , 0.57927869, 0.42304656])

**RMSE Mean:**

In [8]:
RMSE.mean(axis=0)


array([1.3131393 , 0.57927869, 0.42304656])

**RMSE Min:**

In [9]:
RMSE.min(axis=0)

array([0.59950973, 0.20402168, 0.14985824])